In [1]:
from torchvision.models.segmentation import deeplabv3_resnet50
import torch
import torch.nn as nn
import torch.optim as optim
import os
import numpy as np
from PIL import Image
from torchvision import transforms
from sklearn.metrics import jaccard_score
import tarfile
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import functional as F
from albumentations import Compose, Normalize, HorizontalFlip, RandomCrop
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import matplotlib.pyplot as plt
import zipfile

# Путь к архиву
archive_path = 'val.zip'
# Путь, куда распаковать
extract_path = '/content/'

# Проверяем, существует ли папка для извлечения
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Распаковка архива
with zipfile.ZipFile(archive_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Архив разархивирован.")


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Архив разархивирован.


In [3]:
def remove_ds_store_files(directory):
    """
    Удаляет все файлы .DS_Store в указанной директории и её подкаталогах.
    """
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == '.DS_Store':
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Удалён файл: {file_path}")

image_dir = '/content/val/valImages'
remove_ds_store_files(image_dir)

image_dir = '/content/val/valLabels'
remove_ds_store_files(image_dir)


Удалён файл: /content/val/valImages/.DS_Store


In [9]:
import cv2

# --- Параметры ---
IMAGE_DIR = "val/valImages"  # Путь к папке с изображениями
LABEL_DIR = "val/valLabels"  # Путь к папке с масками
BATCH_SIZE = 4
NUM_CLASSES = 2  # Два класса: разметка или нет разметки
NUM_EPOCHS = 25
ACCUMULATION_STEPS = 4  # Для градиентного накопления
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Кастомный Dataset ---
class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_files = self._get_image_files(image_dir)
        self.label_files = self._match_label_files(self.image_files, label_dir)
        self.transform = transform

    def _get_image_files(self, image_dir):
        """
        Возвращает список изображений, игнорируя ненужные файлы, например, .DS_Store.
        """
        image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg')) and not f.startswith('.')]
        return sorted(image_files)

    def _match_label_files(self, image_files, label_dir):
        """
        Фильтрует файлы масок, оставляя только те, которые соответствуют изображениям.
        """
        valid_files = []
        for image_file in image_files:
            label_file = image_file.replace("_prev", "")
            label_path = os.path.join(label_dir, label_file)
            if os.path.exists(label_path):
                valid_files.append(label_file)
        return valid_files

    def __len__(self):
        return len(self.label_files)

    def __getitem__(self, idx):
        label_file = self.label_files[idx]
        image_file = label_file.replace(".png", "_prev.png")

        image_path = os.path.join(self.image_dir, image_file)
        label_path = os.path.join(self.label_dir, label_file)

        try:
            # Загрузка изображения и маски
            image = np.array(Image.open(image_path).convert("RGB"))
            label = np.array(Image.open(label_path))
        except Exception as e:
            print(f"Error loading image or label: {e}")
            return None, None

        # Преобразование маски: выделение только класса (0, 0, 255)
        label = self._convert_to_binary_mask(label)

        # Применение трансформаций
        if self.transform:
            augmented = self.transform(image=image, mask=label)
            image = augmented["image"]
            label = augmented["mask"]

        return image, label.long()

    @staticmethod
    def _convert_to_binary_mask(label):
        """
        Преобразует маску в бинарный формат:
        - Все пиксели с цветом (0, 0, 255) становятся 1 (разметка).
        - Остальные пиксели становятся 0 (фон).
        """
        binary_mask = np.zeros(label.shape[:2], dtype=np.uint8)
        binary_mask[(label[:, :, 0] == 0) & (label[:, :, 1] == 0) & (label[:, :, 2] == 255)] = 1
        return binary_mask
# --- Определение модели ---
def initialize_model(num_classes=2, pretrained=True):
    model = deeplabv3_resnet50(pretrained=pretrained)
    model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=1)
    return model

# --- Функция для расчета IoU ---
def calculate_iou(pred_mask, true_mask):
    # Преобразуем маски в 1D массивы
    pred_mask = pred_mask.flatten()
    true_mask = true_mask.flatten()
    # Используем jaccard_score для расчета IoU
    return jaccard_score(true_mask, pred_mask)

# --- Расчет IoU ---
def evaluate_iou(model, image_dir, label_dir, device):
    model.eval()
    iou_scores = []

    image_files = sorted(os.listdir(image_dir))
    label_files = sorted(os.listdir(label_dir))

    with torch.no_grad():
        for image_file, label_file in zip(image_files, label_files):
            image_path = os.path.join(image_dir, image_file)
            label_path = os.path.join(label_dir, label_file)

            # Загрузка изображения и маски
            image = Image.open(image_path).convert("RGB")
            label = np.array(Image.open(label_path))

            # Преобразуем метку в бинарную маску
            true_mask = CustomDataset._convert_to_binary_mask(label)

            # Применяем преобразования
            transform = transforms.Compose([
                transforms.Resize((2048, 2048)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            input_tensor = transform(image).unsqueeze(0).to(device)

            # Прогнозирование
            with torch.amp.autocast("cuda"):
                output = model(input_tensor)["out"]

            probabilities = torch.sigmoid(output)
            threshold = 0.32
            pred_mask = probabilities[0, 1, :, :].cpu().numpy() > threshold

            # Приводим предсказанную маску к размеру истинной маски
            pred_mask_resized = cv2.resize(pred_mask.astype(np.float32), (true_mask.shape[1], true_mask.shape[0]))
            pred_mask_resized = pred_mask_resized > threshold

            # Вычисление IoU
            iou = calculate_iou(pred_mask_resized, true_mask)
            iou_scores.append(iou)

    # Усредненный IoU
    mean_iou = np.mean(iou_scores)
    return mean_iou

# --- Основная часть кода ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "deeplabv3p_epoch20.pth"  # Путь к модели

# Инициализация модели
model = initialize_model(num_classes=2, pretrained=False)
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)

# Загрузка состояния модели
filtered_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()}
model.load_state_dict(filtered_state_dict, strict=False)

model.to(DEVICE)
model.eval()

# --- Расчет IoU ---
mean_iou = evaluate_iou(model, IMAGE_DIR, LABEL_DIR, DEVICE)
print(f"Средний IoU: {mean_iou:.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead. warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`. warnings.warn(msg)
<ipython-input-8-fb0433df0095>:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions th